In [1]:
import os
import hail as hl
from gwas_analysis.dask import io
from pysnptools import snpreader
import dask.array as da
%run {os.environ['NB_DIR']}/nb.py
%run $BENCHMARK_METHOD_DIR/common.py
sample_rate = .05
ds_name = DATASET_HM
ds_config = DATASET_CONFIG[ds_name]
ds_export_path = dataset_path(ds_name, sample_rate)
hail_init()

Running on Apache Spark version 2.4.4
SparkUI available at http://2e4e0c6972f9:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.32-a5876a0a2853
LOGGING: writing to /home/eczech/repos/gwas-analysis/notebooks/benchmark/method/ld_prune/lsh/hail-20200220-2218-0.2.32-a5876a0a2853.log


In [2]:
mt = hl.import_plink(
    *plink_files(osp.dirname(ds_config['path']), osp.basename(ds_config['path'])),
    skip_invalid_loci=False,
    reference_genome=ds_config['reference_genome']
)
mt = mt.sample_rows(p=sample_rate, seed=1)
mt.count()

2020-02-20 22:18:17 Hail: INFO: Found 165 samples in fam file.
2020-02-20 22:18:17 Hail: INFO: Found 1457897 variants in bim file.
2020-02-20 22:18:24 Hail: INFO: Coerced sorted dataset


(72732, 165)

In [3]:
def export(mt, path):
    hl.export_plink(
        mt, path, 
        fam_id=mt.fam_id,
        pat_id=mt.pat_id,
        mat_id=mt.mat_id,
        is_female=mt.is_female,
        pheno=mt.is_case,
        varid=mt.rsid
    )
export(mt, ds_export_path)
ds_export_path

2020-02-20 22:01:16 Hail: INFO: Coerced sorted dataset
2020-02-20 22:01:21 Hail: INFO: Coerced sorted dataset
2020-02-20 22:01:26 Hail: INFO: Coerced sorted dataset
2020-02-20 22:01:31 Hail: INFO: Coerced sorted dataset
2020-02-20 22:01:36 Hail: INFO: Coerced sorted dataset
2020-02-20 22:01:44 Hail: INFO: merging 3 files totalling 2.9M...
2020-02-20 22:01:44 Hail: INFO: while writing:
    /tmp/benchmark_datasets/hapmap-0.05.bed
  merge time: 33.845ms
2020-02-20 22:01:44 Hail: INFO: merging 2 files totalling 2.1M...
2020-02-20 22:01:44 Hail: INFO: while writing:
    /tmp/benchmark_datasets/hapmap-0.05.bim
  merge time: 26.052ms
2020-02-20 22:01:45 Hail: INFO: merging 16 files totalling 4.0K...
2020-02-20 22:01:45 Hail: INFO: while writing:
    /tmp/benchmark_datasets/hapmap-0.05.fam
  merge time: 10.007ms
2020-02-20 22:01:45 Hail: INFO: wrote 72732 variants and 165 samples to '/tmp/benchmark_datasets/hapmap-0.05'


'/tmp/benchmark_datasets/hapmap-0.05'

In [9]:
client = get_dask_client(n_workers=4)
client

/opt/conda/envs/hail/lib/python3.7/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:37681 Dashboard: http://127.0.0.1:40611/status,Cluster Workers: 4 Cores: 4 Memory: 132.00 GB


In [12]:
gt = da.from_array(io.BedArray(snpreader.Bed(ds_export_path, count_A1=True)), lock=False)
gt

dask.array<array, shape=(72732, 165), dtype=uint8, chunksize=(72732, 165), chunktype=numpy.ndarray>

In [13]:
path = ds_export_path + '.zarr'
gt.to_zarr(path, overwrite=True)
!du -ch $path

'/tmp/benchmark_datasets/hapmap-0.05.zarr'